# Lab: Cross-Validation and the Bootstrap

In this lab, we'll explore the four resampling techniques discussed in Chapter 5 of ISLR: the validation set approach, leave-one-out cross-validation (LOOCV), $k$-fold cross-validation, and the bootstrap. Note that some of the commands in this lab may take a while to run.

## The Validation Set Approach

First, we'll explore using the validation set approach to estimate the test error rates that result from fitting various linear models with the `Auto` data set.

In [1]:
library(ISLR)

Note that before starting, we use the `set.seed()` function in order to set a *seed* for `R`'s random number generator. This is generally a good idea to do when performing analyses that contain an element of randomness, such as cross-validation, in order to have reproducible results.

In [2]:
set.seed(1)

To start with, we use the `sample()` function to split the set of observations into two halves by randomly selecting a training set that consists of 196 of the original 392 observations. Note that in passing the integer 392 as the first argument, `x` in sample, we are using the shortcut that sampling will take place from the vector `1:392`, as described in the documentation.

In [3]:
train = sample(392, 196)

Once we have created the vector to denote our training set, we then use the `subset` option in `lm()` to fit a linear regression model using only the observations corresponding to those in the training set.

In [4]:
lm.fit = lm(mpg ~ horsepower, data = Auto, subset = train)

After fitting the linear regression of `mpg` onto `horsepower` using the training set, we use the `predict()` function to estimate the response for all 392 observations. Then, we use the `mean()` function to compute the mean squared error of the 196 observations in the validation set. Recall that the index `-train` below tells `R` to exclude the observations that are not in the training set. In other words, it selets those observations that are in the validation set.

In [5]:
mean((Auto$mpg - predict(lm.fit, Auto))[-train]^2)

[1] 23.26601

As we can see, the the estimated test mean squared error for the linear regression fit is 23.27. We can then use the `poly()` function to estimate the test error for the quadratic and cubic regressions. (As a side note, don't forget that by default the `poly()` function produces orthogonal polynomials by default. This isn't important for making predictions or estimating the test error, but it is still good to remember, since ISLR doesn't make this fact explicit.)

In [6]:
lm.fit2 = lm(mpg ~ poly(horsepower, 2), data = Auto, subset = train)
mean((Auto$mpg - predict(lm.fit2, Auto))[-train]^2)

[1] 18.71646

In [7]:
lm.fit3 = lm(mpg ~ poly(horsepower, 3), data = Auto, subset = train)
mean((Auto$mpg - predict(lm.fit3, Auto))[-train]^2)

[1] 18.79401

Here we see that the validation set mean squared error rates are 18.72 and 18.79 for the quadratic and cubic fits, respectively. Note that due to the element of randomness in choosing the training set, if we used a different seed (and therefore choose a possibly different training set), we will obtain somewhat different validation set error values.

In [8]:
set.seed(2)
train = sample(392, 196)

In [9]:
lm.fit = lm(mpg ~ horsepower, data = Auto, subset = train)
mean((Auto$mpg - predict(lm.fit, Auto))[-train]^2)

[1] 25.72651

In [10]:
lm.fit2 = lm(mpg ~ poly(horsepower, 2), data = Auto, subset = train)
mean((Auto$mpg - predict(lm.fit2, Auto))[-train]^2)

[1] 20.43036

In [11]:
lm.fit3 = lm(mpg ~ poly(horsepower, 3), data = Auto, subset = train)
mean((Auto$mpg - predict(lm.fit3, Auto))[-train]^2)

[1] 20.38533

Using this training set/validation set split, we get validation set mean squared error values of 25.73, 20.43, and 20.38 for the linear, quadratic, and cubic regression models, respectively. 

These results are consistent with our findings from the previous chapters that used an approach focused more on the statistics of the coefficient estimates: a quadratic model for predicting `mpg` using `horsepower` performs better than a linear model, and there isn't evidence to suggest that using a cubic model provides a meaningful improvement.

## Leave-One-Out Cross Validation

Next up is leave-one-out cross validation (LOOCV). In `R` this can be automatically computed for any generalized linear model using the `glm()` and `cv.glm()` functions. Previously, in Lab 3, we used `glm()` to perform logistic regression by passing the argument `family = "binomial"`. If we don't pass this argument, then by default `glm()` performs linear regression just like the `lm()` function.

In [12]:
glm.fit = glm(mpg ~ horsepower, data = Auto)
coef(glm.fit)

(Intercept)  horsepower 
 39.9358610  -0.1578447

In [13]:
lm.fit = lm(mpg ~ horsepower, data = Auto)
coef(lm.fit)

(Intercept)  horsepower 
 39.9358610  -0.1578447

As we can see, both yield identical linear regression models. For this lab, we will use `glm()` over `lm()` in order to make use of the `cv.glm()` function, which is part of the `boot` library.

In [1]:
library(boot)

In [15]:
glm.fit = glm(mpg ~ horsepower, data = Auto)
cv.err = cv.glm(Auto, glm.fit)
cv.err$delta

[1] 24.23151 24.23114

The `cv.glm()` function produces a list with several components:

- `call`: The original call to `cv.glm()`
- `K`: The value of `K` used for the $k$-fold cross validation. If no value of `K` was supplied to the function call, then this is the number of observations in the matrix or data frame that was passed to the function.
- `delta`: A vector of length two containing the cross-validation results. The first component is the raw cross-validation estimate of prediction error, which is given by the formula below. The second component is the adjusted cross-validation estimate, which uses an adjustment designed to compensate for the bias introduced if $k$-fold cross-validation is used instead of leave-one-out cross validation
- `seed`: The value of `R`'s random seed when `cv.glm()` was called.

Recall that the $k$-fold cross validation estimate for the test mean squared error with $k$ folds is given by

\begin{equation}
    \text{CV}_{(k)} = \frac{1}{k} \sum_{i = 1}^k \text{MSE}_i,
\end{equation}

where $\text{MSE}_i$ is the mean squared error computed on the $i$th held out fold after fitting the model on the remaining $k - 1$ folds. In the case of leave-one-out cross-validation, $k = n$, the number of obserations in the data, and $\text{MSE}_i$ is the mean squared error $(y_i - \hat{y}_i)^2$ obtained after fitting the statistical learning method on all observations except for $y_i$.

Since we used leave-one-out cross-validation in this call to the `cv.glm()` function, the two values in `delta` are equal up to two decimal places, with a value of 24.23.

We can make use of a `for` loop to iteratively repeat the procedure of leave-one-out cross-validation for increasingly complex polynomial fits. We will iteratively fit polynomial regressions for polynomials of order $i = 1, \dots, 10$, compute the associated LOOCV error, and store it in the $i$th element of the vector `cv.error`.

In [16]:
cv.error = rep(0, 10)
for (i in 1:10){
    glm.fit = glm(mpg ~ poly(horsepower, i), data = Auto)
    cv.error[i] = cv.glm(Auto, glm.fit)$delta[1]
}
cv.error

[1] 24.23151 19.24821 19.33498 19.42443 19.03321 18.97864 18.83305 18.96115
 [9] 19.06863 19.49093

As we can see, there is a sharp drop in the estimated test mean squared error between the linear and quadratic fits, but then no clear improvement from using higher-order polynomials. This agrees with Figure 5.4 in ISLR.

## $k$-Fold Cross-Validation

We can also use the `cv.glm()` function to perform $k$-fold cross validation by supplying a value `K` to the function call. We'll use $k = 10$, a common choice for $k$, on the `Auto` data set and again use a `for` loop to iteratively compute the $k$-fold cross validation errors corresponding to polynomial fits of order $i = 1, \dots, 10$. Note that since $k$-fold cross validation involves random sampling, we once again set a random seed to get reproducible results.

In [17]:
set.seed(17)
cv.error = rep(0, 10)
for (i in 1:10){
    glm.fit = glm(mpg ~ poly(horsepower, i), data = Auto)
    cv.error[i] = cv.glm(Auto, glm.fit, K = 10)$delta[1]
}
cv.error

[1] 24.27207 19.26909 19.34805 19.29496 19.03198 18.89781 19.12061 19.14666
 [9] 18.87013 20.95520

Note that the computation time is much shorter than that of LOOCV; the computation was near instantaneous as opposed to taking about 30-45 seconds. While the formula

\begin{equation}
    \text{CV}_{(n)} = \frac{1}{n} \sum_{i = 1}^n \left( \frac{y_i - \hat{y}_i}{1 - h_i} \right)^2,
\end{equation}

where $\hat{y}_i$ is the $i$th fitted value from the original least squares fit, and $h_i$ is the leverage value of the $i$th observation, could be used, in princlple, to greatly speed up the computation of LOOCV in the case of least squares or polynomial regression, `cv.glm()` does not make use of it.

As with before, there is still little evidence that using cubic or higher-order polynomial terms leads to lower test error than simply using a quadratic fit.

## The Bootstrap

To illustrate the use of the bootstrap, first we continue with the example shown in Section 5.2 of ISLR using the simulated `Portfolio` data set before moving on to an example involving estimating the accuracy of the linear regression model on the `Auto` data set.

### Estimating the Accuracy of a Statistic of Interest

One of the strengths of the bootstrap approach is that it is very widely applicable and does not require complicated mathematical complications; in `R`, we only need to take two steps to perform a bootstrap analysis. The first is creating a function that computes the statistic of interest. Second, we use the `boot()` function from the `boot` library to perform the bootstrap by repeatedly sampling observations from the data with replacement.

As already noted, we start by working the the `Portfolio` data set from the `ISLR` package, which described in Section 5.2 of ISLR. It consists of 100 simulated pairs of returns for the investments $X$ and $Y$. We are trying to choose the the fraction of our money $\alpha$ to invest in $X$ (investing the remaining $1 - \alpha$ in $Y$) that minimizes the variance of our investment. In other words, we want to minimize $\text{Var}(\alpha X + (1 - \alpha)Y)$. It can be shown that the value of $\alpha$ which minimizes the risk is given by

\begin{equation}
    \alpha = \frac{\sigma_Y^2 - \sigma_{XY}}{\sigma_X^2 + \sigma_Y^2 - 2\sigma_{XY}},
\end{equation}

where $\sigma_X^2 = \text{Var}(X)$, $\sigma_Y^2 = \text{Var}(Y)$, and $\sigma_{XY} = \text{Cov}(X, Y)$. Since the `Portfolio` data set consists of simulated data, we know that the true values of the parameters were set to $\sigma_X^2 = 1$, $\sigma_Y^2 = 1.25$, and $\sigma_{XY} = 0.5$.

To illustrate our use of the bootstrap on this data, we first create a function, `alpha.fn()`, which takes two arguments:

- `data`: the $(X, Y)$ data
- `index`: a vector indicating which observations should be used to estimate $\alpha$.

The function then outputs the estimate for $\alpha$ based on the selected observations.

In [18]:
alpha.fn = function(data, index){
    X = data$X[index]
    Y = data$Y[index]
    return ((var(Y) - cov(X, Y))/(var(X) + var(Y) - 2*cov(X, Y)))
}

The function returns an estimate for $\alpha$ by plugging in the estimates for $\sigma_X^2 = \text{Var}(X)$, $\sigma_Y^2 = \text{Var}(Y)$, and $\sigma_{XY} = \text{Cov}(X, Y)$ into the above formula for $\alpha$. For example, we can use the function to estimate $\alpha$ using all 100 observations in the `Portfolio` data set.

In [19]:
alpha.fn(Portfolio, 1:100)

[1] 0.5758321

Next, we can use the `sample()` function to randomly select 100 observations, with replacement, from the range 1 to 100. This is equivalent to constructing a new bootstrap data set and recomputing $\hat{\alpha}$ with it.

In [20]:
set.seed(1)
alpha.fn(Portfolio, sample(100, 100, replace = T))

[1] 0.7368375

While we could perform bootstrap analysis by performing this command many times, recording all of the corresponding estimates $\hat{\alpha}$, and computing the resulting standard deviation, it is much more convenient to use the `boot()` function to automate the process. Beow we produce $R = 1000$ bootstrap estimates for $\alpha$.

In [2]:
boot(Portfolio, alpha.fn, R = 1000)


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Portfolio, statistic = alpha.fn, R = 1000)


Bootstrap Statistics :
     original       bias    std. error
t1* 0.5758321 -0.001695873  0.09366347

The final output shows that when bootstrapping with the `Portfolio` data, $\hat{\alpha} = 0.5758$ and that the bootstrap estimate for $\text{SE}(\hat{\alpha})$ is $0.0937$. This is fairly close to the true value $\alpha = 0.6$.

### Estimating the Accuracy of a Linear Regression Model

Next, we'll apply the bootstrap to assess the variability of the coefficient estimates and predictions from a statistical learning method. We'll demonstrate that by using the bootstrap to assess the variability of the estimates of $\beta_0$ and $\beta_1$, the intercept and slope terms for the linear regression model that uses `horsepower` to predict `mpg` in the `Auto` data set. We'll then compare the bootstrap estimates with the ones obtained using the formulas for $\text{SE}(\hat{\beta}_i)$ described in Section 3.1.2 of ISLR.

\begin{equation}
    \text{SE}(\hat{\beta}_0)^2 = \sigma^2 \left[ \frac{1}{n} + \frac{\bar{x}^2}{\sum_{i = 1}^n (x_i - \bar{x})^2} \right], \,
    \text{SE}(\hat{\beta}_1)^2 = \frac{\sigma^2}{\sum_{i = 1}^n (x_i - \bar{x})^2}
\end{equation}

Recall that $\sigma$ is estimated by using the residual standard error (RSE) $\hat{\sigma}^2$ for the model, which is computed using the formula

\begin{equation}
    \text{RSE} = \hat{\sigma} = \sqrt{\frac{1}{n - 2} \sum_{i = 1}^n (y_i - \hat{y}_i)^2}.
\end{equation}

To start, we create the function `boot.fn()` which takes in the `Auto` data set along with a set of indices for the observations, and returns the intercept and slope estimates for the linear regression model. Note that since the function is only one line long, we aren't required to include the curly braces `{` and `}` at the beginning and end of the function.

In [22]:
boot.fn = function(data, index)
    return(coef(lm(mpg ~ horsepower, data = data, subset = index)))

First, we'll demonstrate applying this function to the full set of 392 observations in order to compute the estimates of $\beta_0$ and $\beta_1$ on the entire data set using the usual linear regression coefficient estimate formulas from Chapter 3 of ISLR.

In [23]:
boot.fn(Auto, 1:392)

(Intercept)  horsepower 
 39.9358610  -0.1578447

Just like we did with `alpha.fn()`, we'll use `boot.fn()` to create bootstrap estimates for the intercept and slope terms by randomly sampling from among the observations with replacement. Here are two examples before using the `boot()` function to actually peform the full bootstrap.

In [24]:
set.seed(1)
boot.fn(Auto, sample(392, 392, replace = T))

(Intercept)  horsepower 
 40.3404517  -0.1634868

In [25]:
boot.fn(Auto, sample(392, 392, replace = T))

(Intercept)  horsepower 
 40.1186906  -0.1577063

Now we use the `boot()` function to compute the standard errors of 1,000 bootstrap estimates for the intercept and slope terms.

In [26]:
boot(Auto, boot.fn, R = 1000)


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Auto, statistic = boot.fn, R = 1000)


Bootstrap Statistics :
      original        bias    std. error
t1* 39.9358610  0.0544513229 0.841289790
t2* -0.1578447 -0.0006170901 0.007343073

We got bootstrap estimates of 0.84 for $\text{SE}(\hat{\beta}_0)$ and 0.0073 for $\text{SE}(\hat{\beta}_1)$. Now we'll compare them with the estimates computed using the above formulas, which can be obtained using the `summary()` function.

In [27]:
summary(lm(mpg ~ horsepower, data = Auto))$coef

,Estimate,Std. Error,t value,Pr(>|t|)
(Intercept),39.9358610,0.717498656,55.65984,1.220362e-187
horsepower,-0.1578447,0.006445501,-24.48914,7.031989e-81


Using the formulas from Section 3.1.2, the standard error estimates are 0.717 for the intercept and 0.0064 for the slope, which are somewhat different from the bootstrap estimates. This is due to the assumptions underlying those formulas used for the estimation. 

- First, those formulas depend on the unknown parameter $\sigma^2$, the noise variance, which we estimated using the residual sum of squares. While the formulas for the standard errors of the coefficients don't rely on the correctness of the model, our estimate for $\sigma^2$ does. We saw previously that there is a non-linear relationship `mpg` and `horsepower`, which results in inflated residuals from a linear fit. In turn, this will affect our estimate of $\sigma^2$.
- In addition, the standard formulas make the (somewhat unrealistic) assumption that the $x_i$ are fixed, and that all of the variability comes from the variation in the errors $\epsilon_i$.

The bootstrap approach doesn't rely on any of these assumptions, so it is likely that the bootstrap estimates are more accurate estimates of the standard errors of $\hat{\beta}_0$ and $\hat{\beta}_1$ than those from the `summary()` function.

Next, we use the bootstrap to compute standard error estimates for a quadratic model and compare them with the standard linear regression estimates. Since a quadratic model provides a good fit to the data, there is now a better correpondence between the bootstrap estimates and the standard estimates of $\text{SE}(\hat{\beta}_i)$ for $i = 0, 1, 2$.

In [28]:
boot.fn = function(data, index)
    return(coef(lm(mpg ~ horsepower + I(horsepower^2), data = data, subset = index)))
set.seed(1)
boot(Auto, boot.fn, 1000)


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Auto, statistic = boot.fn, R = 1000)


Bootstrap Statistics :
        original        bias     std. error
t1* 56.900099702  3.511640e-02 2.0300222526
t2* -0.466189630 -7.080834e-04 0.0324241984
t3*  0.001230536  2.840324e-06 0.0001172164

In [29]:
summary(lm(mpg ~ horsepower + I(horsepower^2), data = Auto))$coef

,Estimate,Std. Error,t value,Pr(>|t|)
(Intercept),56.900099702,1.8004268063,31.60367,1.740911e-109
horsepower,-0.466189630,0.0311246171,-14.97816,2.289429e-40
I(horsepower^2),0.001230536,0.0001220759,10.08009,2.196340e-21
